In [7]:
%load_ext autoreload
%autoreload 2
from mtsccleav import *

In [65]:
from aeon.utils.conversion import convert_collection
from aeon.utils.validation import get_type
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from aeon.transformations.collection.convolution_based import Rocket
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


In [9]:
from aeon.classification.convolution_based import (
    Arsenal,
    HydraClassifier,
    MiniRocketClassifier,
    MultiRocketClassifier,
    MultiRocketHydraClassifier,
    RocketClassifier,
)

In [10]:
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score,
    confusion_matrix, matthews_corrcoef
)
import numpy as np

In [11]:
ts_panel = load_as_pd_multiindex("../data/01_single_multi_eq.csv")

In [12]:
ts_panel

five_p_cleav_1  five_p_cleav_compl_1  five_p_cleav_2  \
instances time_points                                                         
0         0                         0                     1              -1   
          1                         1                     0               0   
          2                         0                     1              -1   
          3                         1                     0               0   
          4                        -1                     0               0   
...                               ...                   ...             ...   
826       9                         1                     0               0   
          10                        0                     0               1   
          11                        0                     0              -1   
          12                       -1                     0               0   
          13                        0                    -1              -1   

                       five_p_cleav_compl_2  five_p_cleav_prob  \
instances time_points                                            
0         0                               0           0.999468   
          1                              -1           0.998997   
          2                               0           0.998971   
          3                              -1           0.999318   
          4                               1           0.999742   
...                                     ...                ...   
826       9                               0           0.000000   
          10                              0           0.000000   
          11                              0           0.000000   
          12                              1           0.330473   
          13                              0           0.300819   

                       five_p_cleav_ss  five_p_non_cleav_1  \
instances time_points                                        
0         0                          1                   1   
          1                          1                  -1   
          2                          1                  -1   
          3                          1                   0   
          4                          1                   0   
...                                ...                 ...   
826       9                          0                   0   
          10                         0                   0   
          11                         0                  -1   
          12                         1                   0   
          13                         1                   1   

                       five_p_non_cleav_compl_1  five_p_non_cleav_2  \
instances time_points                                                 
0         0                                   0                   0   
          1                                   0                   0   
          2                                   0                   0   
          3                                   1                  -1   
          4                                   1                  -1   
...                                         ...                 ...   
826       9                                  -1                   1   
          10                                  0                   1   
          11                                  0                   0   
          12                                 -1                   1   
          13                                  0                   0   

                       five_p_non_cleav_compl_2  ...  three_p_cleav_2  \
instances time_points                            ...                    
0         0                                  -1  ...                0   
          1                                   1  ...                0   
          2                                  -1  ...                0   
          3                             

In [13]:
X = construct_X(ts_panel, include_five_p_cleav=True, include_prob=True, include_ss=True)
X 

five_p_cleav_1  five_p_cleav_compl_1  five_p_cleav_2  \
instances time_points                                                         
0         0                         0                     1              -1   
          1                         1                     0               0   
          2                         0                     1              -1   
          3                         1                     0               0   
          4                        -1                     0               0   
...                               ...                   ...             ...   
1653      9                         0                    -1               1   
          10                        0                     0               1   
          11                       -1                     0               0   
          12                        0                    -1               1   
          13                        1                     0               0   

                       five_p_cleav_compl_2  five_p_cleav_prob  \
instances time_points                                            
0         0                               0           0.999468   
          1                              -1           0.998997   
          2                               0           0.998971   
          3                              -1           0.999318   
          4                               1           0.999742   
...                                     ...                ...   
1653      9                               0           0.992391   
          10                              0           0.000000   
          11                              0           0.000000   
          12                              0           0.762377   
          13                             -1           0.705569   

                       five_p_cleav_ss  
instances time_points                   
0         0                          1  
          1                          1  
          2                          1  
          3                          1  
          4                          1  
...                                ...  
1653      9                          1  
          10                         0  
          11                         0  
          12                         1  
          13                         1  

[23156 rows x 6 columns]

In [14]:
X_convert=convert_collection(X, "np-list")

In [15]:
get_type(X), get_type(X_convert)

('pd-multiindex', 'np-list')

In [16]:
y = np.concatenate((np.full((1, 827), 'cleav'), np.full((1, 827), 'non_cleav')), axis=None)
labelencoder = LabelEncoder()
y_transform = labelencoder.fit_transform(y)

In [10]:
y_transform

array([0, 0, 0, ..., 1, 1, 1])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_convert, y_transform, test_size=0.2, random_state=42)

In [12]:
rocket = Rocket(n_kernels=10000)
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)
print(X_train_transform.shape)

(1323, 20000)


In [13]:
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [14]:
X_test_transform = rocket.transform(X_test)
classifier.score(X_test_transform, y_test)

0.8580060422960725

In [15]:
# Using pipeline

In [16]:
rocket = MiniRocketClassifier(n_kernels=1000)
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
accuracy_score(y_test, y_pred)

0.851963746223565

In [2]:
def multiclass_specificity(cm):
    FP = cm.sum(axis=0) - np.diag(cm)
    TN = cm.sum() - (FP + cm.sum(axis=1) - np.diag(cm) + np.diag(cm))
    return np.mean(TN / (TN + FP))

In [37]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [55]:
for fold, (train_idx, test_idx) in enumerate(cv.split(X_convert, y_transform), 1):
    print(f"Fold {fold}")
    X_train = [X_convert[i] for i in train_idx]
    X_test = [X_convert[i] for i in test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]
    print((y_test))
    clf = RocketClassifier()
    clf.fit(X_train, y_train)


Fold 1
['cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cleav'
 'cleav' 'cleav' 'cleav' 'cleav' 'cleav' 'cl

In [82]:
def evaluate_classifier(classifier, X, y, n_splits=5):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.linear_model import RidgeClassifierCV

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    results = {k: [] for k in ["accuracy", "sensitivity", "specificity", "f1", "mcc"]}

    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y), 1):
        print(f"Fold {fold}")

        X_train = [X[i] for i in train_idx]
        X_test = [X[i] for i in test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]
    
        print(y_train)
        # transformer.fit(X_train)
        # X_train_trans = transformer.transform(X_train)
        # X_test_trans = transformer.transform(X_test)

        # clf = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        # clf.fit(X_train_trans, y_train)
        # y_pred = clf.predict(X_test_trans)

        clf = RocketClassifier()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        

        # acc = accuracy_score(y_test, y_pred)
        # sens = recall_score(y_test, y_pred, average="macro")
        # f1 = f1_score(y_test, y_pred, average="macro")
        # mcc = matthews_corrcoef(y_test, y_pred)
        # cm = confusion_matrix(y_test, y_pred)
        # spec = multiclass_specificity(cm)

        # results["accuracy"].append(acc)
        # results["sensitivity"].append(sens)
        # results["specificity"].append(spec)
        # results["f1"].append(f1)
        # results["mcc"].append(mcc)

        sens = recall_score(y_test, y_pred, average="macro")
        print(sens)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        specificity = tn / (tn+fp)
        sensitivity = tp / (tp+fn)
        f1_score = 2 * tp / (2 * tp + fp + fn)
        mcc = matthews_corrcoef(y_test, y_pred)

        print(accuracy, specificity, sensitivity, f1_score, mcc)

        results["accuracy"].append(accuracy)
        results["sensitivity"].append(sensitivity)
        results["specificity"].append(specificity)
        results["f1"].append(f1_score)
        # results["mcc"].append(mcc)

    print("\nAverage 5-Fold Results:")
    for metric, scores in results.items():
        print(f"{metric.capitalize():<12}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

In [33]:

# MiniRocket
print("\n🔹 MiniRocket Evaluation")
evaluate_transformer(MiniRocket(random_state=42), X, y)

# MultiRocket
print("\n🔹 MultiRocket Evaluation")
evaluate_transformer(MultiRocket(random_state=42), X, y)

# Hydra
print("\n🔹 Hydra Evaluation")
evaluate_transformer(Hydra(random_state=42), X, y)


🔹 MiniRocket Evaluation


ValueError: Found input variables with inconsistent numbers of samples: [23156, 1654]

In [20]:
from aeon.classification.convolution_based import (
    MiniRocketClassifier, MultiRocketClassifier, HydraClassifier,
)

In [31]:
y_transform

array([0, 0, 0, ..., 1, 1, 1])

In [83]:
evaluate_classifier(MultiRocketHydraClassifier(random_state=18), X_convert, y_transform)

Fold 1
[0 0 0 ... 1 1 1]
0.845929171230376
0.8459214501510574 0.8433734939759037 0.8484848484848485 0.8459214501510574 0.6918583424607521
Fold 2
[0 0 0 ... 1 1 1]
0.8731653888280394
0.8731117824773413 0.8554216867469879 0.8909090909090909 0.875 0.7467398381294643
Fold 3
[0 0 0 ... 1 1 1]
0.8608981380065718
0.8610271903323263 0.8181818181818182 0.9036144578313253 0.8670520231213873 0.7245793069547013
Fold 4
[0 0 0 ... 1 1 1]
0.8459656809054399
0.8459214501510574 0.8606060606060606 0.8313253012048193 0.8440366972477065 0.6921841221341273
Fold 5
[0 0 0 ... 1 1 1]
0.8636363636363636
0.8636363636363636 0.8181818181818182 0.9090909090909091 0.8695652173913043 0.7302967433402214

Average 5-Fold Results:
Accuracy    : 0.8579 ± 0.0106
Sensitivity : 0.8767 ± 0.0311
Specificity : 0.8392 ± 0.0180
F1          : 0.8603 ± 0.0128
Mcc         : nan ± nan


/Users/cyuab/miniconda3/envs/mtsccleave/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/cyuab/miniconda3/envs/mtsccleave/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/cyuab/miniconda3/envs/mtsccleave/lib/python3.12/site-packages/numpy/_core/_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/cyuab/miniconda3/envs/mtsccleave/lib/python3.12/site-packages/numpy/_core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/cyuab/miniconda3/envs/mtsccleave/lib/python3.12/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
